In [1]:
import numpy as np
import math
import ipywidgets as widgets
from ipycanvas import Canvas, hold_canvas
from time import sleep

In [2]:
CANVAS_HEIGTH = 200
CANVAS_WIDTH = 200
CANVAS_X_OFFSET = CANVAS_WIDTH / 2
CANVAS_Y_OFFSET = CANVAS_HEIGTH / 2
CANVAS_XLIM = (-CANVAS_X_OFFSET, CANVAS_WIDTH-CANVAS_X_OFFSET)
CANVAS_YLIM = (-CANVAS_Y_OFFSET, CANVAS_HEIGTH-CANVAS_Y_OFFSET)

In [3]:
def draw_seg(canvas:Canvas, v1, v2):
    canvas.stroke_line(
        int(v1[0] + CANVAS_X_OFFSET), 
        int(canvas.height - v1[1] - 1 - CANVAS_Y_OFFSET), 
        int(v2[0] + CANVAS_X_OFFSET), 
        int(canvas.height - v2[1]) - 1 - CANVAS_Y_OFFSET)

def draw_vertices(canvas:Canvas, vertices):
    for i in range(1, len(vertices)):
        draw_seg(canvas, vertices[i-1], vertices[i])
        
def rotate_x(vertices, a):
    m = np.array([
        [1, 0, 0, 0],
        [0, math.cos(a), math.sin(a), 0],
        [0, -math.sin(a), math.cos(a), 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

def rotate_y(vertices, a):
    m = np.array([
        [math.cos(a), 0, -math.sin(a), 0],
        [0, 1, 0, 0],
        [math.sin(a), 0, math.cos(a), 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

def rotate_z(vertices, a):
    m = np.array([
        [math.cos(a), math.sin(a), 0, 0],
        [-math.sin(a), math.cos(a), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

def translate(vertices, dx, dy, dz):
    m = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [dx, dy, dz, 1],
    ])
    return vertices @ m

def scale(vertices, sx, sy, sz):
    m = np.array([
        [sx, 0, 0, 0],
        [0, sy, 0, 0],
        [0, 0, sz, 0],
        [0, 0, 0, 1]
    ])
    return vertices @ m

In [4]:
# x y z h, p, y
class Camera:
    def __init__(self, position, heading, pitch, yaw):
        self.position = position
        self.heading = heading
        self.pitch = pitch
        self.yaw = yaw
    def rotate_matrix(self):
        return rotate_z(rotate_x(rotate_y(np.eye(4), -self.heading), -self.pitch), -self.yaw)
    def translate_matrix(self):
        return translate(np.eye(4), -self.position[0], -self.position[1], -self.position[2])
    def view_matrix(self):
        return self.rotate_matrix() @ self.translate_matrix()
camera = Camera([0,0,0], math.radians(0),math.radians(0),math.radians(0))

In [5]:
obj_cube_vertices = np.array([
    [-50, -50, -50, 1],#0
    [-50,  50, -50, 1],#1
    [ 50,  50, -50, 1],#2
    [ 50, -50, -50, 1],#3
    [-50, -50, 50, 1], #4
    [-50,  50, 50, 1], #5
    [ 50,  50, 50, 1], #6
    [ 50, -50, 50, 1]] #7
)
obj_cube_seg_indices = np.array([
    0,1,1,2,2,3,3,0,
    4,5,5,6,6,7,7,4,
    0,4,1,5,2,6,3,7
])

def draw_segments_element(canvas:Canvas, vertices, indices):
    for i in range(0, len(indices), 2):
        draw_seg(canvas, vertices[indices[i]], vertices[indices[i+1]])


# 相机角度变换

In [6]:
canvas = Canvas(height=CANVAS_HEIGTH, width=CANVAS_WIDTH)
display(canvas)

def render():
    with hold_canvas():
        canvas.clear()
        draw_segments_element(canvas, obj_cube_vertices @ camera.view_matrix(), obj_cube_seg_indices)
render()

headingSlider = widgets.FloatSlider(value=0, min=0, max=360, description='heading:')
def headingChanged(evt):
    camera.heading = math.radians(evt.new)
    render()
headingSlider.observe(headingChanged, 'value')

pitchSlider = widgets.FloatSlider(value=0, min=0, max=360, description='pitch:')
def pitchChanged(evt):
    camera.pitch = math.radians(evt.new)
    render()
pitchSlider.observe(pitchChanged, 'value')

yawSlider = widgets.FloatSlider(value=0, min=0, max=360, description='yaw:')
def yewChanged(evt):
    camera.yaw = math.radians(evt.new)
    render()
yawSlider.observe(yewChanged, 'value')

xSlider = widgets.FloatSlider(value=0, min=-200, max=200, description='x:')
def xChanged(evt):
    camera.position[0] = evt.new
    render()
xSlider.observe(xChanged, 'value')

ySlider = widgets.FloatSlider(value=0, min=-200, max=200, description='y:')
def yChanged(evt):
    camera.position[1] = evt.new
    render()
ySlider.observe(yChanged, 'value')

zSlider = widgets.FloatSlider(value=0, min=-200, max=200, description='z:')
def zChanged(evt):
    camera.position[2] = evt.new
    render()
zSlider.observe(zChanged, 'value')

widgets.VBox([headingSlider, pitchSlider, yawSlider, xSlider, ySlider, zSlider])

Canvas(height=200, width=200)